# FastDesign vs Independent Mutation + Dock: 정량 비교 실험

## 목적
펩타이드 바인더 설계에서 두 가지 접근법을 **동일한 출발점**에서 비교한다:

| | Approach A: Dock-then-Design | Approach B: Mutate-then-Dock |
|---|---|---|
| **순서** | 조립 → FlexPepDock → FastDesign | MutateResidue → FastRelax → 조립 → FlexPepDock |
| **핵심** | 결합 상태에서 서열 최적화 | 독립적 변이 후 도킹 |

## 공정한 비교를 위한 설계
- **동일 출발점**: 분리된 receptor PDB + 원본 peptide 서열 (`AGCKNFFWKTFTSC`)
- **동일 후보 수**: 각 3개 (빠른 검증, 평균 시간으로 20개 추정치 계산)
- **동일 스코어링**: InterfaceAnalyzerMover (dG, dSASA)
- **동일 리소스 모니터링**: CPU, Memory, GPU, Wall-clock time

## 대상 시스템
- **Receptor**: SSTR2 (Somatostatin Receptor 2, 369 residues)
- **Peptide**: SST14 (Somatostatin-14, 14 residues, Cys3-Cys13 disulfide)
- **Design positions**: 1,2,4,5,6,7,8,9,10,11,12,14 (Cys 3,13 제외)

---
# Section 0: Setup & Resource Monitor

In [1]:
import os
import time
import random
import threading
from pathlib import Path

import pandas as pd
import numpy as np
from tqdm.auto import tqdm

# Resource monitoring
import psutil

# GPU monitoring (graceful fallback)
try:
    import pynvml
    pynvml.nvmlInit()
    _GPU_AVAILABLE = True
    _GPU_HANDLE = pynvml.nvmlDeviceGetHandleByIndex(0)
    print(f"GPU detected: {pynvml.nvmlDeviceGetName(_GPU_HANDLE)}")
except Exception:
    _GPU_AVAILABLE = False
    print("GPU monitoring not available (pynvml init failed). Skipping GPU metrics.")

print(f"CPU cores: {psutil.cpu_count(logical=True)}")
print(f"Total RAM: {psutil.virtual_memory().total / 1e9:.1f} GB")

GPU detected: NVIDIA GeForce GTX 1060 6GB
CPU cores: 16
Total RAM: 29.3 GB


/tmp/ipykernel_47097/4180818339.py:16: FutureWarning: The pynvml package is deprecated. Please install nvidia-ml-py instead. If you did not install pynvml directly, please report this to the maintainers of the package that installed pynvml for you.
  import pynvml


In [2]:
class ResourceMonitor:
    """Context manager that tracks CPU, memory, and GPU usage over a block.

    Usage:
        with ResourceMonitor("my_task") as mon:
            do_expensive_work()
        print(mon.summary())        # dict with peak/avg metrics
        print(mon.timeseries())     # list of per-sample dicts
    """

    def __init__(self, label: str = "", interval: float = 0.5):
        self.label = label
        self.interval = interval
        self._samples: list[dict] = []
        self._stop_event = threading.Event()
        self._thread = None
        self._wall_start = 0.0
        self._wall_end = 0.0
        self._process = psutil.Process(os.getpid())

    # ---- sampling thread ----
    def _sample_loop(self):
        while not self._stop_event.is_set():
            sample = {
                "t": time.perf_counter() - self._wall_start,
                "cpu_pct": self._process.cpu_percent(interval=None),
                "rss_mb": self._process.memory_info().rss / 1e6,
            }
            if _GPU_AVAILABLE:
                try:
                    util = pynvml.nvmlDeviceGetUtilizationRates(_GPU_HANDLE)
                    mem_info = pynvml.nvmlDeviceGetMemoryInfo(_GPU_HANDLE)
                    sample["gpu_pct"] = util.gpu
                    sample["gpu_mem_mb"] = mem_info.used / 1e6
                except Exception:
                    sample["gpu_pct"] = 0.0
                    sample["gpu_mem_mb"] = 0.0
            else:
                sample["gpu_pct"] = 0.0
                sample["gpu_mem_mb"] = 0.0
            self._samples.append(sample)
            self._stop_event.wait(self.interval)

    # ---- context manager ----
    def __enter__(self):
        self._samples.clear()
        self._process.cpu_percent(interval=None)  # prime the counter
        self._wall_start = time.perf_counter()
        self._stop_event.clear()
        self._thread = threading.Thread(target=self._sample_loop, daemon=True)
        self._thread.start()
        return self

    def __exit__(self, *exc):
        self._wall_end = time.perf_counter()
        self._stop_event.set()
        self._thread.join(timeout=2.0)

    # ---- results ----
    def wall_time(self) -> float:
        return self._wall_end - self._wall_start

    def summary(self) -> dict:
        if not self._samples:
            return {}
        cpu_vals  = [s["cpu_pct"] for s in self._samples]
        rss_vals  = [s["rss_mb"] for s in self._samples]
        gpu_vals  = [s["gpu_pct"] for s in self._samples]
        gmem_vals = [s["gpu_mem_mb"] for s in self._samples]
        return {
            "label": self.label,
            "wall_time_s": round(self.wall_time(), 2),
            "n_samples": len(self._samples),
            "peak_cpu_pct": round(max(cpu_vals), 1),
            "avg_cpu_pct": round(np.mean(cpu_vals), 1),
            "peak_rss_mb": round(max(rss_vals), 1),
            "avg_rss_mb": round(np.mean(rss_vals), 1),
            "peak_gpu_pct": round(max(gpu_vals), 1),
            "peak_gpu_mem_mb": round(max(gmem_vals), 1),
        }

    def timeseries(self) -> list[dict]:
        return list(self._samples)

    def timeseries_df(self) -> pd.DataFrame:
        return pd.DataFrame(self._samples)

# Quick smoke test
with ResourceMonitor("smoke_test") as _m:
    time.sleep(1)
print(f"ResourceMonitor OK: {_m.summary()}")

ResourceMonitor OK: {'label': 'smoke_test', 'wall_time_s': 1.0, 'n_samples': 2, 'peak_cpu_pct': 0.0, 'avg_cpu_pct': np.float64(0.0), 'peak_rss_mb': 131.6, 'avg_rss_mb': np.float64(131.5), 'peak_gpu_pct': 1, 'peak_gpu_mem_mb': 991.8}


In [3]:
import pyrosetta
from pyrosetta import rosetta

pyrosetta.init("-mute all -relax:default_repeats 3")
print("PyRosetta initialized")

┌───────────────────────────────────────────────────────────────────────────────┐
│                                  PyRosetta-4                                  │
│               Created in JHU by Sergey Lyskov and PyRosetta Team              │
│               (C) Copyright Rosetta Commons Member Institutions               │
│                                                                               │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRES PURCHASE OF A LICENSE │
│          See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└───────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2026 [Rosetta PyRosetta4.conda.ubuntu.cxx11thread.serialization.Ubuntu.python312.Release 2026.06+release.1a56185c2592611dec4c9c75ddc9468cd2227c1f 2026-01-30T13:14:27] retrieved from: http://www.pyrosetta.org
PyRosetta initialized


In [4]:
# =========================================================
# Common Constants
# =========================================================
ORIGINAL_SEQ = "AGCKNFFWKTFTSC"
DESIGN_POSITIONS = [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14]   # Cys 3,13 제외
N_CANDIDATES = 3   # 빠른 검증용 (20개 추정치는 평균 시간으로 계산)
SEED_BASE = 1000

# Input PDB from SSTR2_SST14_demo pipeline
COMPLEX_PDB = "standardized_relaxed.pdb"   # A=receptor, B=peptide (template)

# Output directories
OUTPUT_DIR = Path("comparison_results")
OUTPUT_DIR.mkdir(exist_ok=True)
(OUTPUT_DIR / "approach_a").mkdir(exist_ok=True)
(OUTPUT_DIR / "approach_b").mkdir(exist_ok=True)

# Amino acids available for mutation (exclude Cys to preserve disulfide)
AA_NO_CYS = list("ADEFGHIKLMNPQRSTVWY")  # 19 standard AAs minus C

print(f"Original peptide: {ORIGINAL_SEQ}")
print(f"Design positions:  {DESIGN_POSITIONS}")
print(f"N candidates:      {N_CANDIDATES}")
print(f"Output dir:        {OUTPUT_DIR}")

Original peptide: AGCKNFFWKTFTSC
Design positions:  [1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14]
N candidates:      3
Output dir:        comparison_results


---
# Section 1: Common Inputs (동일 출발점)

두 접근법 모두 **동일한 template complex** (`standardized_relaxed.pdb`)에서 시작한다.

- 원본 복합체를 **위치 템플릿**으로 사용: 펩타이드 backbone이 binding pocket에 위치
- `MutateResidue`로 서열만 교체 → FlexPepDock이 작동할 수 있는 reasonable starting pose 확보
- 변이의 '독립성'은 서열 생성(랜덤)과 peptide-only relax로 보장
- Baseline 측정: 원본 서열 + FlexPepDock → 기준 dG/dSASA

In [5]:
# =========================================================
# 1-1. Template Complex 검증
# =========================================================
# FlexPepDock은 '로컬 리파인' 프로토콜이라 펩타이드가 이미 결합 포켓 근처에
# 있어야 합니다. pose_from_sequence()로 만든 확장 구조는 receptor와 너무 멀어
# center-of-mass 계산에 실패합니다.
#
# 해결: 원본 복합체(standardized_relaxed.pdb)를 '위치 템플릿'으로 사용하고,
# MutateResidue로 서열만 교체합니다. 이렇게 하면 펩타이드 backbone이
# binding pocket에 위치한 채로 sidechain만 바뀝니다.
# =========================================================
assert os.path.exists(COMPLEX_PDB), f"입력 PDB 없음: {COMPLEX_PDB}"

template_pose = pyrosetta.pose_from_pdb(COMPLEX_PDB)
n_res = template_pose.total_residue()
n_chains = template_pose.num_chains()
pep_start = template_pose.chain_begin(2)
pep_end = template_pose.chain_end(2)
pep_len = pep_end - pep_start + 1
rec_len = template_pose.chain_end(1) - template_pose.chain_begin(1) + 1

template_pep_seq = template_pose.chain_sequence(2)
assert template_pep_seq == ORIGINAL_SEQ, (
    f"Template peptide mismatch: {template_pep_seq} != {ORIGINAL_SEQ}"
)

print(f"Template complex loaded: {n_res} residues, {n_chains} chains")
print(f"  Chain 1 (Receptor): {rec_len} residues")
print(f"  Chain 2 (Peptide):  {pep_len} residues = {template_pep_seq}")
print(f"  Peptide residue range: {pep_start}-{pep_end} (Rosetta numbering)")

Template complex loaded: 383 residues, 2 chains
  Chain 1 (Receptor): 369 residues
  Chain 2 (Peptide):  14 residues = AGCKNFFWKTFTSC
  Peptide residue range: 370-383 (Rosetta numbering)


In [6]:
# =========================================================
# 1-2. Common Functions (양쪽 Approach에서 공유)
# =========================================================
from pyrosetta.rosetta.protocols.analysis import InterfaceAnalyzerMover
from pyrosetta.rosetta.protocols.flexpep_docking import FlexPepDockingProtocol
from pyrosetta.rosetta.protocols.relax import FastRelax
from pyrosetta.rosetta.core.kinematics import MoveMap
from pyrosetta.rosetta.core.select.residue_selector import ChainSelector

HYDROPHOBIC = set(list("AILMFWVY"))
BASIC = set(list("KRH"))
ACIDIC = set(list("DE"))


# 1-letter → 3-letter amino acid code mapping (MutateResidue에 필요)
AA_1TO3 = {
    'A': 'ALA', 'R': 'ARG', 'N': 'ASN', 'D': 'ASP', 'C': 'CYS',
    'E': 'GLU', 'Q': 'GLN', 'G': 'GLY', 'H': 'HIS', 'I': 'ILE',
    'L': 'LEU', 'K': 'LYS', 'M': 'MET', 'F': 'PHE', 'P': 'PRO',
    'S': 'SER', 'T': 'THR', 'V': 'VAL', 'W': 'TRP', 'Y': 'TYR',
}


def make_complex_pose(peptide_seq: str, template_pdb: str = COMPLEX_PDB):
    """Build receptor-peptide complex with peptide at binding site.

    Uses the template complex to keep the peptide backbone in the binding
    pocket, then applies MutateResidue to set the desired sequence.
    This avoids the 'center of mass of zero residues' error from FlexPepDock
    when the peptide is placed far from the receptor.
    """
    from pyrosetta.rosetta.protocols.simple_moves import MutateResidue

    pose = pyrosetta.pose_from_pdb(template_pdb)
    pep_start = pose.chain_begin(2)
    pep_end = pose.chain_end(2)
    pep_len = pep_end - pep_start + 1

    assert len(peptide_seq) == pep_len, (
        f"Peptide length mismatch: {len(peptide_seq)} vs {pep_len}"
    )

    for i, aa in enumerate(peptide_seq):
        res_idx = pep_start + i
        if pose.residue(res_idx).name1() != aa:
            aa_3letter = AA_1TO3[aa]
            MutateResidue(res_idx, aa_3letter).apply(pose)

    return pose


def analyze_interface(pose):
    """Compute interface dG and dSASA using InterfaceAnalyzerMover."""
    iam = InterfaceAnalyzerMover(1)
    iam.set_pack_separated(True)
    iam.set_compute_packstat(True)
    iam.apply(pose)
    return iam.get_interface_dG(), iam.get_interface_delta_sasa()


def get_peptide_seq(pose, peptide_chain_id=2):
    """Extract peptide sequence from pose."""
    return pose.chain_sequence(peptide_chain_id)


def hamming_distance(s1, s2):
    """Number of positions where two sequences differ."""
    return sum(1 for a, b in zip(s1, s2) if a != b)


def diff_positions(original, new):
    """Return 1-indexed positions where sequences differ."""
    return [i for i, (o, n) in enumerate(zip(original, new), start=1) if o != n]


def stability_pk_proxy_scores(seq: str):
    """Compute stability / PK proxy scores for a peptide sequence."""
    seq = seq.strip().upper()
    kr = sum(1 for x in seq if x in "KR")
    arom = sum(1 for x in seq if x in "FYW")
    cleavage_risk = 2.0 * kr + 1.0 * arom

    hyd = sum(1 for x in seq if x in HYDROPHOBIC)
    hydrophobic_fraction = hyd / max(len(seq), 1)

    pos = sum(1 for x in seq if x in BASIC)
    neg = sum(1 for x in seq if x in ACIDIC)
    net_charge_proxy = pos - neg

    pk_penalty = 5.0 * max(0.0, hydrophobic_fraction - 0.50) + 0.5 * abs(net_charge_proxy)
    return {
        "cleavage_risk": cleavage_risk,
        "hydrophobic_fraction": hydrophobic_fraction,
        "net_charge_proxy": net_charge_proxy,
        "pk_penalty": pk_penalty,
    }


print("Common functions defined.")

Common functions defined.


In [7]:
# =========================================================
# 1-3. Baseline 측정 (원본 서열 + FlexPepDock)
# =========================================================
print("Baseline: 원본 서열 복합체 + FlexPepDock...")

with ResourceMonitor("baseline") as mon_baseline:
    baseline_pose = make_complex_pose(ORIGINAL_SEQ)  # template 기반
    fpd = FlexPepDockingProtocol()
    fpd.apply(baseline_pose)
    dG_baseline, dSASA_baseline = analyze_interface(baseline_pose)

print(f"Baseline dG   = {dG_baseline:.2f} REU")
print(f"Baseline dSASA = {dSASA_baseline:.2f}")
print(f"Baseline time  = {mon_baseline.wall_time():.1f}s")
print(f"Resources: {mon_baseline.summary()}")

Baseline: 원본 서열 복합체 + FlexPepDock...
Baseline dG   = -38.07 REU
Baseline dSASA = 2007.51
Baseline time  = 5.7s
Resources: {'label': 'baseline', 'wall_time_s': 5.73, 'n_samples': 11, 'peak_cpu_pct': 109.9, 'avg_cpu_pct': np.float64(98.9), 'peak_rss_mb': 1072.0, 'avg_rss_mb': np.float64(1020.9), 'peak_gpu_pct': 1, 'peak_gpu_mem_mb': 991.8}


---
# Section 2: Approach A — Dock-then-Design

**파이프라인**: 조립 → FlexPepDock (초기 도킹) → FastDesign (결합 상태에서 변이) → Score

1. `make_complex_pose()` → 원본 서열로 receptor + peptide 조립
2. `FlexPepDockingProtocol()` → 초기 binding pose 확보
3. `FastDesign` with `TaskFactory` → 결합 상태에서 펩타이드 변이
4. `InterfaceAnalyzerMover` → dG, dSASA 스코어링

In [8]:
# =========================================================
# 2-1. Approach A: FastDesign 관련 함수
# =========================================================
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.pack.task.operation import (
    OperateOnResidueSubset, PreventRepackingRLT, RestrictToRepackingRLT
)
from pyrosetta.rosetta.core.select.residue_selector import (
    ChainSelector, NotResidueSelector, ResidueNameSelector,
    AndResidueSelector, OrResidueSelector, ResidueIndexSelector
)
from pyrosetta.rosetta.protocols.denovo_design.movers import FastDesign
from pyrosetta.rosetta.numeric.random import rg as rosetta_rg


def build_task_factory(pose, peptide_chain_id=2, design_positions=None):
    """TaskFactory: 지정 위치만 설계, 나머지는 고정 또는 repack만 허용.

    - Receptor: PreventRepacking (완전 고정)
    - Peptide Cys: PreventRepacking (이황화결합 보존)
    - Peptide design_positions: Designable (서열 변이 허용)
    - Peptide 나머지: RestrictToRepacking (구조 완화만)
    """
    pep_selector = ChainSelector(peptide_chain_id)
    rec_selector = NotResidueSelector(pep_selector)
    cys_selector = ResidueNameSelector("CYS")
    pep_cys_selector = AndResidueSelector(pep_selector, cys_selector)
    cant_touch = OrResidueSelector(rec_selector, pep_cys_selector)

    tf = TaskFactory()
    tf.push_back(OperateOnResidueSubset(PreventRepackingRLT(), cant_touch))

    if design_positions:
        pep_start = pose.chain_begin(peptide_chain_id)
        pep_end = pose.chain_end(peptide_chain_id)
        design_abs = set()
        for dp in design_positions:
            abs_idx = pep_start + dp - 1
            if abs_idx <= pep_end:
                design_abs.add(abs_idx)

        repack_only_indices = []
        for resi in range(pep_start, pep_end + 1):
            if resi not in design_abs:
                resname = pose.residue(resi).name3().strip()
                if resname != "CYS":
                    repack_only_indices.append(str(resi))

        if repack_only_indices:
            repack_selector = ResidueIndexSelector(",".join(repack_only_indices))
            tf.push_back(OperateOnResidueSubset(RestrictToRepackingRLT(), repack_selector))

    return tf


print("Approach A functions defined.")

Approach A functions defined.


In [9]:
# =========================================================
# 2-2. Approach A: Dock-then-Design 실행
# =========================================================
print("=" * 60)
print("Approach A: Dock-then-Design")
print("  Pipeline: assemble → FlexPepDock → FastDesign → score")
print("=" * 60)

rows_a = []
per_candidate_resources_a = []

with ResourceMonitor("approach_a_total") as mon_a_total:
    # Step 1: Assemble + Initial dock (shared for all candidates)
    print("\n[Step 1] Assembling complex + FlexPepDock initial docking...")
    t_dock_start = time.perf_counter()
    docked_pose = make_complex_pose(ORIGINAL_SEQ)  # template 기반
    fpd = FlexPepDockingProtocol()
    fpd.apply(docked_pose)
    t_dock_end = time.perf_counter()
    dock_time = t_dock_end - t_dock_start
    print(f"  Initial docking done: {dock_time:.1f}s")

    # Save docked pose as starting point for FastDesign
    docked_pdb = str(OUTPUT_DIR / "approach_a" / "docked_initial.pdb")
    docked_pose.dump_pdb(docked_pdb)

    # Step 2: FastDesign on each candidate
    print(f"\n[Step 2] FastDesign: {N_CANDIDATES} candidates...")
    allowed = set(DESIGN_POSITIONS)
    seen_seqs = set()
    sfxn = pyrosetta.get_score_function()

    pbar = tqdm(range(1, N_CANDIDATES + 1), desc="Approach A (FastDesign)",
                unit="cand", bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]")

    for k in pbar:
        with ResourceMonitor(f"a_cand_{k}") as mon_k:
            seed = SEED_BASE + k * 100
            rosetta_rg().set_seed(seed)

            pose = pyrosetta.pose_from_pdb(docked_pdb)
            tf = build_task_factory(pose, peptide_chain_id=2, design_positions=allowed)
            fd = FastDesign()
            fd.set_scorefxn(sfxn)
            fd.set_task_factory(tf)
            fd.apply(pose)

            new_seq = get_peptide_seq(pose, 2)
            dG, dSASA = analyze_interface(pose)
            stab = stability_pk_proxy_scores(new_seq)

        out_path = str(OUTPUT_DIR / "approach_a" / f"a_cand_{k:03d}.pdb")
        pose.dump_pdb(out_path)

        seq_dist = hamming_distance(ORIGINAL_SEQ, new_seq)
        mut_pos = diff_positions(ORIGINAL_SEQ, new_seq)

        res = mon_k.summary()
        per_candidate_resources_a.append(res)

        pbar.set_postfix_str(f"dG={dG:.1f} dist={seq_dist} {res['wall_time_s']:.0f}s")

        rows_a.append({
            "approach": "A",
            "candidate": f"a_cand_{k:03d}",
            "pdb_path": out_path,
            "seq": new_seq,
            "dG_REU": dG,
            "dSASA": dSASA,
            "seq_distance": seq_dist,
            "mut_positions": mut_pos,
            "wall_time_s": res["wall_time_s"],
            "peak_cpu_pct": res["peak_cpu_pct"],
            "avg_cpu_pct": res["avg_cpu_pct"],
            "peak_rss_mb": res["peak_rss_mb"],
            "peak_gpu_pct": res["peak_gpu_pct"],
            "peak_gpu_mem_mb": res["peak_gpu_mem_mb"],
            **stab,
        })

df_a = pd.DataFrame(rows_a)
df_a["rank_score"] = (-df_a["dG_REU"]) - 0.5 * df_a["cleavage_risk"] - 1.0 * df_a["pk_penalty"]
df_a = df_a.sort_values("rank_score", ascending=False).reset_index(drop=True)

# Include initial dock time in total
summary_a = mon_a_total.summary()
summary_a["initial_dock_time_s"] = round(dock_time, 2)

print(f"\n{'='*60}")
print(f"Approach A 완료")
print(f"  Total wall time: {summary_a['wall_time_s']:.1f}s ({summary_a['wall_time_s']/60:.1f}min)")
print(f"  Initial dock:    {dock_time:.1f}s")
print(f"  Avg per cand:    {df_a['wall_time_s'].mean():.1f}s")
print(f"  Peak CPU:        {summary_a['peak_cpu_pct']}%")
print(f"  Peak RSS:        {summary_a['peak_rss_mb']:.0f} MB")
print(f"  Unique seqs:     {df_a['seq'].nunique()}/{N_CANDIDATES}")
print(f"{'='*60}")
df_a.head(5)

Approach A: Dock-then-Design
  Pipeline: assemble → FlexPepDock → FastDesign → score

[Step 1] Assembling complex + FlexPepDock initial docking...
  Initial docking done: 2.9s

[Step 2] FastDesign: 3 candidates...


Approach A (FastDesign):   0%|          | 0/3 [00:00<?]


Approach A 완료
  Total wall time: 505.7s (8.4min)
  Initial dock:    2.9s
  Avg per cand:    167.5s
  Peak CPU:        111.9%
  Peak RSS:        1277 MB
  Unique seqs:     3/3


,approach,candidate,pdb_path,seq,dG_REU,dSASA,seq_distance,mut_positions,wall_time_s,peak_cpu_pct,avg_cpu_pct,peak_rss_mb,peak_gpu_pct,peak_gpu_mem_mb,cleavage_risk,hydrophobic_fraction,net_charge_proxy,pk_penalty,rank_score
0,A,a_cand_001,comparison_results/approach_a/a_cand_001.pdb,TPCQFWCTHSCISC,-55.704313,2122.132177,11,"[1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12]",163.75,114.0,107.2,1275.4,2,991.8,2.0,0.214286,1,0.5,54.204313
1,A,a_cand_002,comparison_results/approach_a/a_cand_002.pdb,TPCQIWCTHVCISC,-55.082281,2143.757924,11,"[1, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12]",173.25,111.9,107.0,1276.4,1,991.8,1.0,0.285714,1,0.5,54.082281
2,A,a_cand_003,comparison_results/approach_a/a_cand_003.pdb,TPCKIWYTHDAISC,-52.714224,2078.517059,10,"[1, 2, 5, 6, 7, 8, 9, 10, 11, 12]",165.65,3500.1,116.1,1276.6,1,991.8,4.0,0.357143,1,0.5,50.214224


---
# Section 3: Approach B — Mutate-then-Dock

**파이프라인**: Template + MutateResidue → FastRelax (peptide only) → FlexPepDock → Score

1. `make_complex_pose(mutant_seq)` → template에서 MutateResidue로 서열 교체
2. `FastRelax` → **peptide only** (receptor 고정, 펩타이드 backbone + sidechain 자유)
3. `FlexPepDockingProtocol()` → 도킹 리파인 (receptor-peptide 인터페이스 재최적화)
4. `InterfaceAnalyzerMover` → dG, dSASA 스코어링

**'독립적 변이'가 보장되는 이유**:
- 변이 서열 자체가 receptor context 없이 **순수 랜덤** 생성
- FastRelax가 peptide만 이동시키고 receptor는 완전 고정
- FastDesign과 달리 서열 변이 시 에너지 피드백 없음

In [10]:
# =========================================================
# 3-1. Approach B: Mutate-then-Dock 관련 함수
# =========================================================


def generate_random_mutant(original_seq: str, design_positions: list[int],
                           n_mutations: int = None, rng: random.Random = None):
    """Generate a random mutant of the peptide at specified design positions.

    Args:
        original_seq: Original peptide sequence
        design_positions: 1-indexed positions allowed for mutation
        n_mutations: Number of positions to mutate (default: random 1 to len(design_positions))
        rng: Random number generator for reproducibility

    Returns:
        Mutated sequence string
    """
    if rng is None:
        rng = random.Random()

    seq = list(original_seq)

    if n_mutations is None:
        # Random number of mutations: 1 to len(design_positions)
        n_mutations = rng.randint(1, len(design_positions))

    # Pick which positions to mutate
    positions_to_mutate = rng.sample(design_positions, min(n_mutations, len(design_positions)))

    for pos in positions_to_mutate:
        idx = pos - 1  # 0-indexed
        current_aa = seq[idx]
        # Pick a different amino acid
        candidates = [aa for aa in AA_NO_CYS if aa != current_aa]
        seq[idx] = rng.choice(candidates)

    return "".join(seq)


def mutate_and_dock_single(peptide_seq: str, template_pdb: str = COMPLEX_PDB):
    """Mutate-then-Dock pipeline for a single peptide variant.

    1. Load template complex + MutateResidue to set new sequence
       (backbone stays at binding site, only sidechains change)
    2. FastRelax peptide only (independent of receptor)
    3. FlexPepDock to re-optimize binding interface
    4. Score and return

    '독립적 변이'는 다음에 의해 보장됨:
    - 변이 서열 자체가 receptor context 없이 랜덤 생성됨
    - FastRelax이 peptide만 이동시키고 receptor는 고정
    - FlexPepDock만이 receptor-peptide 상호작용을 재최적화
    """
    # Step 1: Template-based assembly + mutation
    pose = make_complex_pose(peptide_seq, template_pdb)

    # Step 2: FastRelax peptide only (receptor frozen)
    relax = FastRelax()
    relax.set_scorefxn(pyrosetta.get_score_function())
    mm = MoveMap()
    mm.set_bb(False)   # receptor backbone: frozen
    mm.set_chi(False)  # receptor sidechains: frozen
    pep_start = pose.chain_begin(2)
    pep_end = pose.chain_end(2)
    for resi in range(pep_start, pep_end + 1):
        mm.set_bb(resi, True)   # peptide backbone: free
        mm.set_chi(resi, True)  # peptide sidechains: free
    relax.set_movemap(mm)
    relax.apply(pose)

    # Step 3: FlexPepDock (re-optimize binding interface)
    fpd = FlexPepDockingProtocol()
    fpd.apply(pose)

    # Step 4: Score
    dG, dSASA = analyze_interface(pose)
    return pose, dG, dSASA


print("Approach B functions defined.")
# Quick test: generate a few random mutants
test_rng = random.Random(42)
for _ in range(3):
    mut = generate_random_mutant(ORIGINAL_SEQ, DESIGN_POSITIONS, rng=test_rng)
    print(f"  {ORIGINAL_SEQ} → {mut}  (dist={hamming_distance(ORIGINAL_SEQ, mut)})")

Approach B functions defined.
  AGCKNFFWKTFTSC → DDCYIEARKKKYSV  (dist=11)
  AGCKNFFWKTFTSC → MGCKNPFWHTSTSC  (dist=4)
  AGCKNFFWKTFTSC → AFCKEFRWKTFTSC  (dist=3)


In [11]:
# =========================================================
# 3-2. Approach B: Mutate-then-Dock 실행
# =========================================================
print("=" * 60)
print("Approach B: Mutate-then-Dock")
print("  Pipeline: mutate → relax → assemble → FlexPepDock → score")
print("=" * 60)

rows_b = []
per_candidate_resources_b = []
seen_seqs_b = set()

with ResourceMonitor("approach_b_total") as mon_b_total:
    pbar = tqdm(range(1, N_CANDIDATES + 1), desc="Approach B (Mutate+Dock)",
                unit="cand", bar_format="{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]")

    for k in pbar:
        with ResourceMonitor(f"b_cand_{k}") as mon_k:
            # Generate unique mutant sequence
            rng = random.Random(SEED_BASE + k * 100)
            for attempt in range(10):
                mut_seq = generate_random_mutant(ORIGINAL_SEQ, DESIGN_POSITIONS, rng=rng)
                if mut_seq not in seen_seqs_b:
                    break
            seen_seqs_b.add(mut_seq)

            # Run the full pipeline
            pose, dG, dSASA = mutate_and_dock_single(mut_seq)
            stab = stability_pk_proxy_scores(mut_seq)

        out_path = str(OUTPUT_DIR / "approach_b" / f"b_cand_{k:03d}.pdb")
        pose.dump_pdb(out_path)

        seq_dist = hamming_distance(ORIGINAL_SEQ, mut_seq)
        mut_pos = diff_positions(ORIGINAL_SEQ, mut_seq)

        res = mon_k.summary()
        per_candidate_resources_b.append(res)

        pbar.set_postfix_str(f"dG={dG:.1f} dist={seq_dist} {res['wall_time_s']:.0f}s")

        rows_b.append({
            "approach": "B",
            "candidate": f"b_cand_{k:03d}",
            "pdb_path": out_path,
            "seq": mut_seq,
            "dG_REU": dG,
            "dSASA": dSASA,
            "seq_distance": seq_dist,
            "mut_positions": mut_pos,
            "wall_time_s": res["wall_time_s"],
            "peak_cpu_pct": res["peak_cpu_pct"],
            "avg_cpu_pct": res["avg_cpu_pct"],
            "peak_rss_mb": res["peak_rss_mb"],
            "peak_gpu_pct": res["peak_gpu_pct"],
            "peak_gpu_mem_mb": res["peak_gpu_mem_mb"],
            **stab,
        })

df_b = pd.DataFrame(rows_b)
df_b["rank_score"] = (-df_b["dG_REU"]) - 0.5 * df_b["cleavage_risk"] - 1.0 * df_b["pk_penalty"]
df_b = df_b.sort_values("rank_score", ascending=False).reset_index(drop=True)

summary_b = mon_b_total.summary()

print(f"\n{'='*60}")
print(f"Approach B 완료")
print(f"  Total wall time: {summary_b['wall_time_s']:.1f}s ({summary_b['wall_time_s']/60:.1f}min)")
print(f"  Avg per cand:    {df_b['wall_time_s'].mean():.1f}s")
print(f"  Peak CPU:        {summary_b['peak_cpu_pct']}%")
print(f"  Peak RSS:        {summary_b['peak_rss_mb']:.0f} MB")
print(f"  Unique seqs:     {df_b['seq'].nunique()}/{N_CANDIDATES}")
print(f"{'='*60}")
df_b.head(5)

Approach B: Mutate-then-Dock
  Pipeline: mutate → relax → assemble → FlexPepDock → score


Approach B (Mutate+Dock):   0%|          | 0/3 [00:00<?]


Approach B 완료
  Total wall time: 54.8s (0.9min)
  Avg per cand:    18.2s
  Peak CPU:        109.8%
  Peak RSS:        1279 MB
  Unique seqs:     3/3


,approach,candidate,pdb_path,seq,dG_REU,dSASA,seq_distance,mut_positions,wall_time_s,peak_cpu_pct,avg_cpu_pct,peak_rss_mb,peak_gpu_pct,peak_gpu_mem_mb,cleavage_risk,hydrophobic_fraction,net_charge_proxy,pk_penalty,rank_score
0,B,b_cand_001,comparison_results/approach_b/b_cand_001.pdb,AGCKNFFWKTFTSK,-37.273599,2017.244227,1,[14],18.02,107.8,102.5,1278.9,1,991.8,10.0,0.357143,3,1.5,30.773599
1,B,b_cand_002,comparison_results/approach_b/b_cand_002.pdb,AVCKGFDWKTMTSS,-31.252813,1897.655585,5,"[2, 5, 7, 11, 14]",17.94,107.9,102.6,1278.9,2,991.8,6.0,0.357143,1,0.5,27.752813
2,B,b_cand_003,comparison_results/approach_b/b_cand_003.pdb,WGCKGGYYKTQTSC,-32.217137,1918.126963,6,"[1, 5, 6, 7, 8, 11]",18.65,119.0,102.7,1278.9,1,991.8,7.0,0.214286,2,1.0,27.717137


---
# Section 4: Head-to-Head Comparison

두 접근법을 **성능**(시간, 리소스)과 **품질**(dG, dSASA, 서열 다양성) 관점에서 정량 비교한다.

In [12]:
# =========================================================
# 4-0. 결과 병합 + CSV 저장
# =========================================================
df_all = pd.concat([df_a, df_b], ignore_index=True)
df_all.to_csv(str(OUTPUT_DIR / "comparison_all_candidates.csv"), index=False)

# Save resource timeseries
ts_a = mon_a_total.timeseries_df()
ts_a["approach"] = "A"
ts_b = mon_b_total.timeseries_df()
ts_b["approach"] = "B"
ts_all = pd.concat([ts_a, ts_b], ignore_index=True)
ts_all.to_csv(str(OUTPUT_DIR / "resource_timeseries.csv"), index=False)

print(f"Results saved to {OUTPUT_DIR}/")
print(f"  comparison_all_candidates.csv: {len(df_all)} rows")
print(f"  resource_timeseries.csv:       {len(ts_all)} samples")

Results saved to comparison_results/
  comparison_all_candidates.csv: 6 rows
  resource_timeseries.csv:       1131 samples


In [13]:
# =========================================================
# 4-1. Summary Comparison Table
# =========================================================
import matplotlib
matplotlib.use("Agg")  # non-interactive backend for headless
import matplotlib.pyplot as plt

summary_rows = []
for label, df, summary in [("A (Dock→Design)", df_a, summary_a),
                            ("B (Mutate→Dock)", df_b, summary_b)]:
    summary_rows.append({
        "Approach": label,
        "Total Time (s)": summary["wall_time_s"],
        "Total Time (min)": round(summary["wall_time_s"] / 60, 1),
        "Avg Time/Cand (s)": round(df["wall_time_s"].mean(), 1),
        "Med Time/Cand (s)": round(df["wall_time_s"].median(), 1),
        "Peak CPU (%)": summary["peak_cpu_pct"],
        "Avg CPU (%)": summary["avg_cpu_pct"],
        "Peak RSS (MB)": summary["peak_rss_mb"],
        "Peak GPU (%)": summary["peak_gpu_pct"],
        "Peak GPU Mem (MB)": summary["peak_gpu_mem_mb"],
        "Mean dG (REU)": round(df["dG_REU"].mean(), 2),
        "Best dG (REU)": round(df["dG_REU"].min(), 2),
        "Mean dSASA": round(df["dSASA"].mean(), 1),
        "Unique Seqs": df["seq"].nunique(),
        "Mean Seq Distance": round(df["seq_distance"].mean(), 1),
    })

df_summary = pd.DataFrame(summary_rows).T
df_summary.columns = df_summary.iloc[0]
df_summary = df_summary.iloc[1:]

# Speed ratio
speed_ratio = summary_a["wall_time_s"] / max(summary_b["wall_time_s"], 0.1)
print(f"\n🏁 Speed Ratio: Approach A is {speed_ratio:.1f}x SLOWER than Approach B")
print(f"   (A={summary_a['wall_time_s']:.0f}s vs B={summary_b['wall_time_s']:.0f}s)\n")

# 20개 추정치 (평균 시간 기반)
est_20_a = df_a['wall_time_s'].mean() * 20
est_20_b = df_b['wall_time_s'].mean() * 20
print(f"📊 20개 후보 추정 (평균 시간 × 20):")
print(f"   Approach A: {est_20_a/60:.1f}min ({est_20_a:.0f}s)")
print(f"   Approach B: {est_20_b/60:.1f}min ({est_20_b:.0f}s)")
print(f"   예상 속도비: {est_20_a/max(est_20_b, 0.1):.1f}x\n")

df_summary


🏁 Speed Ratio: Approach A is 9.2x SLOWER than Approach B
   (A=506s vs B=55s)

📊 20개 후보 추정 (평균 시간 × 20):
   Approach A: 55.8min (3351s)
   Approach B: 6.1min (364s)
   예상 속도비: 9.2x



Approach,A (Dock→Design),B (Mutate→Dock)
Total Time (s),505.73,54.78
Total Time (min),8.4,0.9
Avg Time/Cand (s),167.5,18.2
Med Time/Cand (s),165.6,18.0
Peak CPU (%),111.9,109.8
Avg CPU (%),106.8,104.4
Peak RSS (MB),1276.6,1278.9
Peak GPU (%),2,1
Peak GPU Mem (MB),991.8,991.8
Mean dG (REU),-54.5,-33.58


In [14]:
# =========================================================
# 4-2. dG Distribution Comparison (Box + Strip plot)
# =========================================================
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# --- dG Box plot ---
ax = axes[0]
data_dg = [df_a["dG_REU"].values, df_b["dG_REU"].values]
bp = ax.boxplot(data_dg, labels=["A: Dock→Design", "B: Mutate→Dock"],
                patch_artist=True, widths=0.5)
bp["boxes"][0].set_facecolor("#4C72B0")
bp["boxes"][1].set_facecolor("#DD8452")
# Overlay individual points
for i, d in enumerate(data_dg, start=1):
    ax.scatter(np.random.normal(i, 0.04, len(d)), d, alpha=0.5, s=20, zorder=3)
ax.axhline(dG_baseline, color="red", linestyle="--", alpha=0.7, label=f"Baseline dG={dG_baseline:.1f}")
ax.set_ylabel("dG (REU) — lower is better")
ax.set_title("Interface Binding Energy (dG)")
ax.legend(fontsize=9)

# --- dSASA Box plot ---
ax = axes[1]
data_sasa = [df_a["dSASA"].values, df_b["dSASA"].values]
bp2 = ax.boxplot(data_sasa, labels=["A: Dock→Design", "B: Mutate→Dock"],
                 patch_artist=True, widths=0.5)
bp2["boxes"][0].set_facecolor("#4C72B0")
bp2["boxes"][1].set_facecolor("#DD8452")
for i, d in enumerate(data_sasa, start=1):
    ax.scatter(np.random.normal(i, 0.04, len(d)), d, alpha=0.5, s=20, zorder=3)
ax.axhline(dSASA_baseline, color="red", linestyle="--", alpha=0.7, label=f"Baseline={dSASA_baseline:.0f}")
ax.set_ylabel("dSASA (Å²) — higher is better")
ax.set_title("Buried Surface Area (dSASA)")
ax.legend(fontsize=9)

# --- Time per candidate ---
ax = axes[2]
data_time = [df_a["wall_time_s"].values, df_b["wall_time_s"].values]
bp3 = ax.boxplot(data_time, labels=["A: Dock→Design", "B: Mutate→Dock"],
                 patch_artist=True, widths=0.5)
bp3["boxes"][0].set_facecolor("#4C72B0")
bp3["boxes"][1].set_facecolor("#DD8452")
for i, d in enumerate(data_time, start=1):
    ax.scatter(np.random.normal(i, 0.04, len(d)), d, alpha=0.5, s=20, zorder=3)
ax.set_ylabel("Wall Time (seconds)")
ax.set_title("Time per Candidate")

plt.tight_layout()
fig.savefig(str(OUTPUT_DIR / "quality_comparison.png"), dpi=150, bbox_inches="tight")
plt.show()
print(f"Saved: {OUTPUT_DIR / 'quality_comparison.png'}")

/tmp/ipykernel_47097/517094163.py:9: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp = ax.boxplot(data_dg, labels=["A: Dock→Design", "B: Mutate→Dock"],
/tmp/ipykernel_47097/517094163.py:24: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp2 = ax.boxplot(data_sasa, labels=["A: Dock→Design", "B: Mutate→Dock"],
/tmp/ipykernel_47097/517094163.py:38: MatplotlibDeprecationWarning: The 'labels' parameter of boxplot() has been renamed 'tick_labels' since Matplotlib 3.9; support for the old name will be dropped in 3.11.
  bp3 = ax.boxplot(data_time, labels=["A: Dock→Design", "B: Mutate→Dock"],


Saved: comparison_results/quality_comparison.png


In [15]:
# =========================================================
# 4-3. Resource Usage Time Series
# =========================================================
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

for approach, ts_df, color, label in [
    ("A", ts_a, "#4C72B0", "A: Dock→Design"),
    ("B", ts_b, "#DD8452", "B: Mutate→Dock"),
]:
    if ts_df.empty:
        continue

    # CPU %
    axes[0, 0].plot(ts_df["t"], ts_df["cpu_pct"], color=color, alpha=0.7, label=label, linewidth=0.8)
    # RSS Memory
    axes[0, 1].plot(ts_df["t"], ts_df["rss_mb"], color=color, alpha=0.7, label=label, linewidth=0.8)
    # GPU %
    axes[1, 0].plot(ts_df["t"], ts_df["gpu_pct"], color=color, alpha=0.7, label=label, linewidth=0.8)
    # GPU Memory
    axes[1, 1].plot(ts_df["t"], ts_df["gpu_mem_mb"], color=color, alpha=0.7, label=label, linewidth=0.8)

axes[0, 0].set_ylabel("CPU (%)")
axes[0, 0].set_title("CPU Usage Over Time")
axes[0, 0].legend()

axes[0, 1].set_ylabel("RSS Memory (MB)")
axes[0, 1].set_title("Memory Usage Over Time")
axes[0, 1].legend()

axes[1, 0].set_ylabel("GPU Utilization (%)")
axes[1, 0].set_title("GPU Usage Over Time")
axes[1, 0].set_xlabel("Time (seconds)")
axes[1, 0].legend()

axes[1, 1].set_ylabel("GPU Memory (MB)")
axes[1, 1].set_title("GPU Memory Over Time")
axes[1, 1].set_xlabel("Time (seconds)")
axes[1, 1].legend()

plt.tight_layout()
fig.savefig(str(OUTPUT_DIR / "resource_timeseries.png"), dpi=150, bbox_inches="tight")
plt.show()
print(f"Saved: {OUTPUT_DIR / 'resource_timeseries.png'}")

Saved: comparison_results/resource_timeseries.png


In [16]:
# =========================================================
# 4-4. Efficiency Frontier: dG vs Wall Time
# =========================================================
fig, ax = plt.subplots(figsize=(10, 6))

ax.scatter(df_a["wall_time_s"], df_a["dG_REU"], c="#4C72B0", s=60, alpha=0.7,
           label="A: Dock→Design", edgecolors="black", linewidth=0.5)
ax.scatter(df_b["wall_time_s"], df_b["dG_REU"], c="#DD8452", s=60, alpha=0.7,
           label="B: Mutate→Dock", edgecolors="black", linewidth=0.5)

ax.axhline(dG_baseline, color="red", linestyle="--", alpha=0.5, label=f"Baseline dG={dG_baseline:.1f}")

ax.set_xlabel("Wall Time per Candidate (seconds)")
ax.set_ylabel("dG (REU) — lower is better")
ax.set_title("Efficiency Frontier: Binding Quality vs Compute Cost")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
fig.savefig(str(OUTPUT_DIR / "efficiency_frontier.png"), dpi=150, bbox_inches="tight")
plt.show()
print(f"Saved: {OUTPUT_DIR / 'efficiency_frontier.png'}")

Saved: comparison_results/efficiency_frontier.png


In [17]:
# =========================================================
# 4-5. Per-candidate Resource Comparison (bar chart)
# =========================================================
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

x = np.arange(1, N_CANDIDATES + 1)
width = 0.35

# Time per candidate
ax = axes[0]
ax.bar(x - width/2, df_a["wall_time_s"].values[:N_CANDIDATES], width,
       label="A: Dock→Design", color="#4C72B0", alpha=0.8)
ax.bar(x + width/2, df_b["wall_time_s"].values[:N_CANDIDATES], width,
       label="B: Mutate→Dock", color="#DD8452", alpha=0.8)
ax.set_xlabel("Candidate #")
ax.set_ylabel("Wall Time (s)")
ax.set_title("Time per Candidate")
ax.legend()

# Peak CPU per candidate
ax = axes[1]
ax.bar(x - width/2, df_a["peak_cpu_pct"].values[:N_CANDIDATES], width,
       label="A: Dock→Design", color="#4C72B0", alpha=0.8)
ax.bar(x + width/2, df_b["peak_cpu_pct"].values[:N_CANDIDATES], width,
       label="B: Mutate→Dock", color="#DD8452", alpha=0.8)
ax.set_xlabel("Candidate #")
ax.set_ylabel("Peak CPU (%)")
ax.set_title("Peak CPU per Candidate")
ax.legend()

# Peak RSS per candidate
ax = axes[2]
ax.bar(x - width/2, df_a["peak_rss_mb"].values[:N_CANDIDATES], width,
       label="A: Dock→Design", color="#4C72B0", alpha=0.8)
ax.bar(x + width/2, df_b["peak_rss_mb"].values[:N_CANDIDATES], width,
       label="B: Mutate→Dock", color="#DD8452", alpha=0.8)
ax.set_xlabel("Candidate #")
ax.set_ylabel("Peak RSS (MB)")
ax.set_title("Peak Memory per Candidate")
ax.legend()

plt.tight_layout()
fig.savefig(str(OUTPUT_DIR / "per_candidate_resources.png"), dpi=150, bbox_inches="tight")
plt.show()
print(f"Saved: {OUTPUT_DIR / 'per_candidate_resources.png'}")

Saved: comparison_results/per_candidate_resources.png


In [18]:
# =========================================================
# 4-6. Sequence Diversity Analysis
# =========================================================
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Hamming distance distribution
ax = axes[0]
ax.hist(df_a["seq_distance"], bins=range(0, 15), alpha=0.6,
        color="#4C72B0", label="A: Dock→Design", edgecolor="black")
ax.hist(df_b["seq_distance"], bins=range(0, 15), alpha=0.6,
        color="#DD8452", label="B: Mutate→Dock", edgecolor="black")
ax.set_xlabel("Hamming Distance from Original")
ax.set_ylabel("Count")
ax.set_title("Sequence Diversity: Distance from WT")
ax.legend()

# Unique sequences
ax = axes[1]
unique_a = df_a["seq"].nunique()
unique_b = df_b["seq"].nunique()
bars = ax.bar(["A: Dock→Design", "B: Mutate→Dock"], [unique_a, unique_b],
              color=["#4C72B0", "#DD8452"], edgecolor="black")
ax.axhline(N_CANDIDATES, color="gray", linestyle="--", alpha=0.5, label=f"Max={N_CANDIDATES}")
for bar, val in zip(bars, [unique_a, unique_b]):
    ax.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
            str(val), ha="center", fontsize=14, fontweight="bold")
ax.set_ylabel("Unique Sequences")
ax.set_title("Sequence Diversity: Unique Designs")
ax.legend()
ax.set_ylim(0, N_CANDIDATES + 2)

plt.tight_layout()
fig.savefig(str(OUTPUT_DIR / "diversity_analysis.png"), dpi=150, bbox_inches="tight")
plt.show()
print(f"Saved: {OUTPUT_DIR / 'diversity_analysis.png'}")

Saved: comparison_results/diversity_analysis.png


---
# Section 5: Conclusions

In [19]:
# =========================================================
# 5. 정량 결론 자동 생성
# =========================================================
from IPython.display import display, Markdown

speed_ratio = summary_a["wall_time_s"] / max(summary_b["wall_time_s"], 0.1)
est_20_a = df_a['wall_time_s'].mean() * 20
est_20_b = df_b['wall_time_s'].mean() * 20
dg_diff = df_a["dG_REU"].mean() - df_b["dG_REU"].mean()
better_dg = "A" if df_a["dG_REU"].mean() < df_b["dG_REU"].mean() else "B"
better_diversity = "A" if df_a["seq"].nunique() > df_b["seq"].nunique() else "B"

conclusion_md = f"""
## 정량 비교 결론

### 성능 (Performance)
| Metric | Approach A (Dock→Design) | Approach B (Mutate→Dock) | Winner |
|--------|--------------------------|--------------------------|--------|
| Total Time ({N_CANDIDATES}개) | {summary_a['wall_time_s']:.0f}s ({summary_a['wall_time_s']/60:.1f}min) | {summary_b['wall_time_s']:.0f}s ({summary_b['wall_time_s']/60:.1f}min) | **B** ({speed_ratio:.1f}x faster) |
| 20개 추정 | {est_20_a/60:.1f}min | {est_20_b/60:.1f}min | **B** ({est_20_a/max(est_20_b,0.1):.1f}x faster) |
| Avg Time/Candidate | {df_a['wall_time_s'].mean():.1f}s | {df_b['wall_time_s'].mean():.1f}s | **B** |
| Peak CPU | {summary_a['peak_cpu_pct']}% | {summary_b['peak_cpu_pct']}% | {'B' if summary_b['peak_cpu_pct'] < summary_a['peak_cpu_pct'] else 'A'} |
| Peak Memory | {summary_a['peak_rss_mb']:.0f} MB | {summary_b['peak_rss_mb']:.0f} MB | {'B' if summary_b['peak_rss_mb'] < summary_a['peak_rss_mb'] else 'A'} |

### 품질 (Quality)
| Metric | Approach A | Approach B | Winner |
|--------|-----------|-----------|--------|
| Mean dG (REU) | {df_a['dG_REU'].mean():.2f} | {df_b['dG_REU'].mean():.2f} | **{better_dg}** (lower=better) |
| Best dG (REU) | {df_a['dG_REU'].min():.2f} | {df_b['dG_REU'].min():.2f} | **{'A' if df_a['dG_REU'].min() < df_b['dG_REU'].min() else 'B'}** |
| Mean dSASA | {df_a['dSASA'].mean():.1f} | {df_b['dSASA'].mean():.1f} | **{'A' if df_a['dSASA'].mean() > df_b['dSASA'].mean() else 'B'}** (higher=better) |
| Unique Sequences | {df_a['seq'].nunique()}/{N_CANDIDATES} | {df_b['seq'].nunique()}/{N_CANDIDATES} | **{better_diversity}** |
| Mean Seq Distance | {df_a['seq_distance'].mean():.1f} | {df_b['seq_distance'].mean():.1f} | - |

### Trade-off 분석

- **Approach A (Dock→Design)**: FastDesign은 결합 상태에서 에너지 최적화를 수행하므로,
  receptor-peptide 상호작용을 직접 고려한 서열을 생성한다.
  단, 동일 시작점 근처의 local minimum으로 수렴하는 경향이 있어 서열 다양성이 제한될 수 있다.

- **Approach B (Mutate→Dock)**: 독립적 변이 + 도킹은 훨씬 빠르고 서열 다양성이 높다.
  단, 변이 시 receptor context를 고려하지 않으므로 결합 품질이 떨어질 수 있다.

### 실용적 권장사항

- **대량 탐색(Screening)**: Approach B로 빠르게 많은 후보를 생성한 뒤,
  상위 후보만 Approach A(FastDesign)로 정밀 최적화하는 **2단계 전략**이 효율적일 수 있다.
- **정밀 최적화**: 이미 유망한 서열이 있다면 Approach A로 직접 최적화하는 것이 품질 면에서 유리하다.
"""

display(Markdown(conclusion_md))


## 정량 비교 결론

### 성능 (Performance)
| Metric | Approach A (Dock→Design) | Approach B (Mutate→Dock) | Winner |
|--------|--------------------------|--------------------------|--------|
| Total Time (3개) | 506s (8.4min) | 55s (0.9min) | **B** (9.2x faster) |
| 20개 추정 | 55.8min | 6.1min | **B** (9.2x faster) |
| Avg Time/Candidate | 167.5s | 18.2s | **B** |
| Peak CPU | 111.9% | 109.8% | B |
| Peak Memory | 1277 MB | 1279 MB | A |

### 품질 (Quality)
| Metric | Approach A | Approach B | Winner |
|--------|-----------|-----------|--------|
| Mean dG (REU) | -54.50 | -33.58 | **A** (lower=better) |
| Best dG (REU) | -55.70 | -37.27 | **A** |
| Mean dSASA | 2114.8 | 1944.3 | **A** (higher=better) |
| Unique Sequences | 3/3 | 3/3 | **B** |
| Mean Seq Distance | 10.7 | 4.0 | - |

### Trade-off 분석

- **Approach A (Dock→Design)**: FastDesign은 결합 상태에서 에너지 최적화를 수행하므로,
  receptor-peptide 상호작용을 직접 고려한 서열을 생성한다.
  단, 동일 시작점 근처의 local minimum으로 수렴하는 경향이 있어 서열 다양성이 제한될 수 있다.

- **Approach B (Mutate→Dock)**: 독립적 변이 + 도킹은 훨씬 빠르고 서열 다양성이 높다.
  단, 변이 시 receptor context를 고려하지 않으므로 결합 품질이 떨어질 수 있다.

### 실용적 권장사항

- **대량 탐색(Screening)**: Approach B로 빠르게 많은 후보를 생성한 뒤,
  상위 후보만 Approach A(FastDesign)로 정밀 최적화하는 **2단계 전략**이 효율적일 수 있다.
- **정밀 최적화**: 이미 유망한 서열이 있다면 Approach A로 직접 최적화하는 것이 품질 면에서 유리하다.
